# Exploratory Data Analysis: Walkthrough
Today we will be demonstrating the following key exploratory data analysis techniques using an example dataset:

**Agenda:**
1. Importing libraries & packages
2. Importing tabular data to a DataFrame
3. Inspecting DataFrame structure
4. Concatenation
5. Renaming Columns
6. Exploring values
7. Exporting DataFrames
8. Merging
9. Plotting



## The Data
Our example dataset is daily summaries of air quality data from Providence, RI. It will give you some experience with working with temporal data.

The Rhode Island Department of Environmental Management (RIDEM) and Rhode Island Department of Health (RIDOH) collects air quality data at several sites across Rhode Island. We will be examining data from one site at the Community of Rhode Island (CCRI) Liston Campus. Here's some background:

* The CCRI site is part of the EPA's *State or Local Air Monitoring Stations* (SLAMS) and *National Air Toxics Trends Sites* (NATTS) networks.
* A variety of air pollutants (particulate matter (PM), volatile organic carbon (VOCs),  polycyclic aromatic hydrocarbons (PAHs), carbonyls, black carbon) have been monitored at this site since 2005.
* A reference for some of the dataset's [field descriptions](https://aqs.epa.gov/aqsweb/airdata/FileFormats.html#_daily_summary_files).
* The data was obtained from the Environmental Protection Agency (EPA) [Air Quality Data website](https://www.epa.gov/outdoor-air-quality-data).

<table><tr>
<td> <img src="images/aq-site-info.png" alt="Drawing" style="width: 400px;"/> </td>
<td> <img src="images/aq-site-table.png" alt="Drawing" style="width: 450px;"/> </td>
</tr></table>

![](./images/aq-site-gearth-dist.png)

We will use a subset of this data in the demonstrations below and give you a chance to work with a larger dataset during the hands-on lab.

*Links*

* [EPA Air Quality Data Interactive Map](https://www.epa.gov/outdoor-air-quality-data/interactive-map-air-quality-monitors) - Data source
* [RIDEM 2022 Annual Monitoring Report](https://dem.ri.gov/sites/g/files/xkgbur861/files/2023-01/airnet22.pdf) - More information about the site and other monitoring locations across the state.
* [National Air Toxics Tends Sites Quality Assurance Project Plan](https://www3.epa.gov/ttnamti1/files/ambient/airtox/NATTS-UATMP-PAMS-SNOC-Analytical-Support-QAPP-2019.pdf) - Detailed measurement guidelines for the toxins in this dataset
* [National Ambient Air Quality Standards](https://www.epa.gov/criteria-air-pollutants/naaqs-table)

---

## 1. Importing libraries & packages
Importing packages typically appears at the top of the file.
* `install.packages("<package_name>")` installs the package. Only needed once. We've set up the JupyterHub to include packages in our environment. So you'll likely only need the following to load your package.
* `library(<package_name>)` loads the package into your R session

In [205]:
library(dplyr)
library(tidyr)
library(readr)

## 2. Importing tabular data to a DataFrame
The pandas package reads tabular data into a data structure called a `DataFrame`. 

### Structure of a DataFrame
![series](./images/pandas_df.png)
Source: Edited from [geeksforgeeks.org](https://www.geeksforgeeks.org/python-pandas-dataframe/)

![series](./images/df_axes.jpeg)

Source: [stackoverflow](https://stackoverflow.com/questions/25773245/ambiguity-in-pandas-dataframe-numpy-array-axis-definition)


### You can read data from many types of formats:
* [`read_csv("path/to/your/file.csv")`](https://readr.tidyverse.org/reference/read_delim.html) - Comma-delimited or other delimited files
* Other common formats include Excel files, Text files, JSON files, SQL databases, and XML files. You'll need different packages and functions to import those formats.

We will be working with the `read_csv()` because our data is comma-delimited. This function defaults to read comma-delimited files, but can be used on any delimited text file when the seperator is specified.

A. To start we need specify the path to our data directory:
```
project
├── data
│   └── raw
│       └── monthly   <- Data is here
│
└── notebooks         <- Our working directory is here
```
We will be using package `os` and `Path` from `pathlib` to create out directory path because it standardizes pathing between operating systems. Path separators are different between Unix (Mac & Linux; using `/`) and Windows (using `\`) operating systems. Avoiding full string paths makes the code universal.

In [206]:
# Create a path to the data directory. The `..` means go up one level from the current working directory
data_path <- file.path("..", "data")

## We extend the path to the monthly data directory
path_to_monthly_data = file.path(data_path, 'raw', 'monthly')

print(paste('This is the monthly data directory: ', path_to_monthly_data))

[1] "This is the monthly data directory:  ../data/raw/monthly"


Using the path generated above, we will read the first month of data (January 2022).

In [207]:
# Read and save the DataFrame object to a variable 'df_2022_01'
df_2022_01 <- read_csv(file.path(path_to_monthly_data, 'daily_44_007_0022_2022_01.csv'))

Rows: 743 Columns: 34
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (14): Datum, Parameter Name, Duration Description, Pollutant Standard, ...
dbl  (14): State Code, County Code, Site Number, Parameter Code, POC, Latitu...
lgl   (5): Exceptional Data Type, Nonreg Observation Count, Nonreg Arithmeti...
date  (1): Date (Local)

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [208]:
head(df_2022_01)

State Code,County Code,Site Number,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Duration Description,⋯,AQI,Daily Criteria Indicator,Tribe Name,State Name,County Name,City Name,Local Site Name,Address,MSA or CBSA Name,Data Source
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
44,7,22,87101,1,41.80747,-71.41297,NAD83,"Particle Number, Total Count",1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,61107,1,41.80747,-71.41297,NAD83,Std Dev Vt Wind Direction,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,62101,1,41.80747,-71.41297,NAD83,Outdoor Temperature,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,61104,1,41.80747,-71.41297,NAD83,Wind Direction - Resultant,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,84313,1,41.80747,-71.41297,NAD83,Black carbon PM2.5 STP,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,88101,3,41.80747,-71.41297,NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,⋯,30,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart


## 3. Inspecting DataFrame Structure
Now that we have imported the data to a DataFrame. Some questions we are curious about:
1. Did it import correctly?
2. What does the table look like? Number of rows? Columns?
3. Do we need all the data we are importing?
4. Is the data in the correct format?

We can inspect the DataFrame object by looking at its **attributes** and using DataFrame **methods**.

Here are useful **attributes** of the dataframe
* `names`:  The names of the columns in the data frame
* `row.names`:  The names of the rows in the data frame
* `class`:  The class of the object, which is typically `data.frame` for data frames
* `dim`: The dimensions of the dataframe

Here are a few useful **methods** to inspect a dataframe:
* `str()`: Shows a listing of the variables and thier variable types
* `attributes()`: Shows a listing of attributes such as name, row.name, class, and dim
* `nrow()`: Shows the length of the data frame
* `ncol()`: Shows the number of columns in the data frame
* `head()`: Shows the first 6 rows of the data by default, but you can specify a different number of rows
* `tail()`: Shows the last 6 rows of the data by default, bur you can specify a different number of rows


In [209]:
# Let's take a look at the head of the data frame!
head(df_2022_01)

State Code,County Code,Site Number,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Duration Description,⋯,AQI,Daily Criteria Indicator,Tribe Name,State Name,County Name,City Name,Local Site Name,Address,MSA or CBSA Name,Data Source
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
44,7,22,87101,1,41.80747,-71.41297,NAD83,"Particle Number, Total Count",1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,61107,1,41.80747,-71.41297,NAD83,Std Dev Vt Wind Direction,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,62101,1,41.80747,-71.41297,NAD83,Outdoor Temperature,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,61104,1,41.80747,-71.41297,NAD83,Wind Direction - Resultant,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,84313,1,41.80747,-71.41297,NAD83,Black carbon PM2.5 STP,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,88101,3,41.80747,-71.41297,NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,⋯,30,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart


In [210]:
# Let's take a look at the variables and thier variable types
str(df_2022_01)

spc_tbl_ [743 × 34] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ State Code                : num [1:743] 44 44 44 44 44 44 44 44 44 44 ...
 $ County Code               : num [1:743] 7 7 7 7 7 7 7 7 7 7 ...
 $ Site Number               : num [1:743] 22 22 22 22 22 22 22 22 22 22 ...
 $ Parameter Code            : num [1:743] 87101 61107 62101 61104 84313 ...
 $ POC                       : num [1:743] 1 1 1 1 1 3 3 3 1 1 ...
 $ Latitude                  : num [1:743] 41.8 41.8 41.8 41.8 41.8 ...
 $ Longitude                 : num [1:743] -71.4 -71.4 -71.4 -71.4 -71.4 ...
 $ Datum                     : chr [1:743] "NAD83" "NAD83" "NAD83" "NAD83" ...
 $ Parameter Name            : chr [1:743] "Particle Number, Total Count" "Std Dev Vt Wind Direction" "Outdoor Temperature" "Wind Direction - Resultant" ...
 $ Duration Description      : chr [1:743] "1 HOUR" "1 HOUR" "1 HOUR" "1 HOUR" ...
 $ Pollutant Standard        : chr [1:743] NA NA NA NA ...
 $ Date (Local)              : Date[1:743], form

<div class="alert alert-block alert-info">
Some observations:

This is a *long format* dataframe. Each row is composed of a date-parameter pair. See below, there are multiple parameters associated with the same date.

A *wide format* dataframe would have each row as a single date and measurements for each parameter would be in individual columns.
</div>



In [211]:
# Highlighting the long format by filtering the columns and getting the head of the data frame
long_data <- df_2022_01 %>% 
    select("Parameter Name", "Date (Local)", "Observation Count") %>%
    head()

long_data

Parameter Name,Date (Local),Observation Count
<chr>,<date>,<dbl>
"Particle Number, Total Count",2022-01-01,24
Std Dev Vt Wind Direction,2022-01-01,24
Outdoor Temperature,2022-01-01,24
Wind Direction - Resultant,2022-01-01,24
Black carbon PM2.5 STP,2022-01-01,24
PM2.5 - Local Conditions,2022-01-01,1


In [212]:
# This is what it would look like as wide data wherein each row is a single date and the measurement for each parameter is in individual columns
pivot_wider(long_data, names_from = "Parameter Name", values_from = "Observation Count")

Date (Local),"Particle Number, Total Count",Std Dev Vt Wind Direction,Outdoor Temperature,Wind Direction - Resultant,Black carbon PM2.5 STP,PM2.5 - Local Conditions
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022-01-01,24,24,24,24,24,1


In [213]:
# Onwards to looking at the tail of the data
tail(df_2022_01)

State Code,County Code,Site Number,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Duration Description,⋯,AQI,Daily Criteria Indicator,Tribe Name,State Name,County Name,City Name,Local Site Name,Address,MSA or CBSA Name,Data Source
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
44,7,22,88101,3,41.80747,-71.41297,NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,⋯,57,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,62201,1,41.80747,-71.41297,NAD83,Relative Humidity,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,84313,1,41.80747,-71.41297,NAD83,Black carbon PM2.5 STP,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,61107,1,41.80747,-71.41297,NAD83,Std Dev Vt Wind Direction,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,61103,1,41.80747,-71.41297,NAD83,Wind Speed - Resultant,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,61104,1,41.80747,-71.41297,NAD83,Wind Direction - Resultant,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart


In [214]:
# Let's get the dimensions
dim(df_2022_01)

[1] 743  34

In [215]:
# Column names
names(df_2022_01)

[1] "State Code"                 "County Code"               
 [3] "Site Number"                "Parameter Code"            
 [5] "POC"                        "Latitude"                  
 [7] "Longitude"                  "Datum"                     
 [9] "Parameter Name"             "Duration Description"      
[11] "Pollutant Standard"         "Date (Local)"              
[13] "Year"                       "Day In Year (Local)"       
[15] "Units of Measure"           "Exceptional Data Type"     
[17] "Nonreg Observation Count"   "Observation Count"         
[19] "Observation Percent"        "Nonreg Arithmetic Mean"    
[21] "Arithmetic Mean"            "Nonreg First Maximum Value"
[23] "First Maximum Value"        "First Maximum Hour"        
[25] "AQI"                        "Daily Criteria Indicator"  
[27] "Tribe Name"                 "State Name"                
[29] "County Name"                "City Name"                 
[31] "Local Site Name"            "Address"                   
[33] "MSA or CBSA Name"           "Data Source"

In [216]:
# Inspect Numerical Fields
df_2022_01 %>%
    select_if(is.numeric) %>%
    head(5)

State Code,County Code,Site Number,Parameter Code,POC,Latitude,Longitude,Year,Day In Year (Local),Observation Count,Observation Percent,Arithmetic Mean,First Maximum Value,First Maximum Hour
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
44,7,22,87101,1,41.80747,-71.41297,2022,1,24,100,7062.208333,14300.00,17
44,7,22,61107,1,41.80747,-71.41297,2022,1,24,100,17.166667,25.00,7
44,7,22,62101,1,41.80747,-71.41297,2022,1,24,100,48.958333,54.00,15
44,7,22,61104,1,41.80747,-71.41297,2022,1,24,100,140.791667,195.00,15
44,7,22,84313,1,41.80747,-71.41297,2022,1,24,100,0.458333,1.25,1


In [217]:
# Inspect Character Fields
df_2022_01 %>%
    select_if(is.character) %>%
    head()

Datum,Parameter Name,Duration Description,Pollutant Standard,Units of Measure,AQI,Daily Criteria Indicator,State Name,County Name,City Name,Local Site Name,Address,MSA or CBSA Name,Data Source
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NAD83,"Particle Number, Total Count",1 HOUR,NA,Count per cm^3,.,Y,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
NAD83,Std Dev Vt Wind Direction,1 HOUR,NA,Degrees Compass,.,Y,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
NAD83,Outdoor Temperature,1 HOUR,NA,Degrees Fahrenheit,.,Y,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
NAD83,Wind Direction - Resultant,1 HOUR,NA,Degrees Compass,.,Y,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
NAD83,Black carbon PM2.5 STP,1 HOUR,NA,Micrograms/cubic meter (25 C),.,Y,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,PM25 Annual 2012,Micrograms/cubic meter (LC),30,Y,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart


**Back to our questions:**

1. Did it import correctly?
2. What does the table look like? Number of rows? Columns?
3. Do we need all the data we are importing?
4. Is the data in the correct format?

* There are many columns we could drop because they all have the same value such as: "Local Site Name" and "Address". We know we are only working with one site for this analysis so these columns don't provide much value. These columns are long string fields that take up more memory. Dropping them would improve performance if this dataset gets really large.
* The date would be more useful as a datetime data type rather than as string. This will allow for filtering by time and other useful datetime operations.
* The column names, while R converted them into valid variable names, are not in a more conventional format like snake case


In [218]:
install.packages('snakecase', repos="http://cran.services.brown.edu")

Warning message:
“unable to access index for repository http://cran.services.brown.edu/src/contrib:
  cannot open URL 'http://cran.services.brown.edu/src/contrib/PACKAGES'”
Warning message:
“package ‘snakecase’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


In [219]:
library(snakecase)

In [220]:
# Clean up the column names
df_2022_01_curated <- df_2022_01 %>%
    rename_with(to_snake_case)

# Print out
head(df_2022_01_curated)
names(df_2022_01_curated)

state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter_name,duration_description,⋯,aqi,daily_criteria_indicator,tribe_name,state_name,county_name,city_name,local_site_name,address,msa_or_cbsa_name,data_source
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
44,7,22,87101,1,41.80747,-71.41297,NAD83,"Particle Number, Total Count",1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,61107,1,41.80747,-71.41297,NAD83,Std Dev Vt Wind Direction,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,62101,1,41.80747,-71.41297,NAD83,Outdoor Temperature,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,61104,1,41.80747,-71.41297,NAD83,Wind Direction - Resultant,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,84313,1,41.80747,-71.41297,NAD83,Black carbon PM2.5 STP,1 HOUR,⋯,.,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart
44,7,22,88101,3,41.80747,-71.41297,NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,⋯,30,Y,NA,Rhode Island,Providence,Providence,CCRI Liston Campus ROOFTOP,"1 Hilton St, PROVIDENCE RI","Providence-Warwick, RI-MA",AQS Data Mart


[1] "state_code"                 "county_code"               
 [3] "site_number"                "parameter_code"            
 [5] "poc"                        "latitude"                  
 [7] "longitude"                  "datum"                     
 [9] "parameter_name"             "duration_description"      
[11] "pollutant_standard"         "date_local"                
[13] "year"                       "day_in_year_local"         
[15] "units_of_measure"           "exceptional_data_type"     
[17] "nonreg_observation_count"   "observation_count"         
[19] "observation_percent"        "nonreg_arithmetic_mean"    
[21] "arithmetic_mean"            "nonreg_first_maximum_value"
[23] "first_maximum_value"        "first_maximum_hour"        
[25] "aqi"                        "daily_criteria_indicator"  
[27] "tribe_name"                 "state_name"                
[29] "county_name"                "city_name"                 
[31] "local_site_name"            "address"                   
[33] "msa_or_cbsa_name"           "data_source"

In [221]:
# Create a list of the columns we wish to keep
keep_cols <- c(
    "parameter_code",
    "poc",
    "parameter_name",
    "duration_description",
    "pollutant_standard",
    "date_local",
    "year",
    "day_in_year_local",
    "units_of_measure",
    "exceptional_data_type",
    "observation_count",
    "observation_percent",
    "arithmetic_mean",
    "first_maximum_value",
    "first_maximum_hour",
    "aqi",
    "daily_criteria_indicator")

df_2022_01_curated <- df_2022_01_curated %>% select(all_of(keep_cols))
head(df_2022_01_curated)

parameter_code,poc,parameter_name,duration_description,pollutant_standard,date_local,year,day_in_year_local,units_of_measure,exceptional_data_type,observation_count,observation_percent,arithmetic_mean,first_maximum_value,first_maximum_hour,aqi,daily_criteria_indicator
<dbl>,<dbl>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
87101,1,"Particle Number, Total Count",1 HOUR,NA,2022-01-01,2022,1,Count per cm^3,NA,24,100,7062.208333,14300.00,17,.,Y
61107,1,Std Dev Vt Wind Direction,1 HOUR,NA,2022-01-01,2022,1,Degrees Compass,NA,24,100,17.166667,25.00,7,.,Y
62101,1,Outdoor Temperature,1 HOUR,NA,2022-01-01,2022,1,Degrees Fahrenheit,NA,24,100,48.958333,54.00,15,.,Y
61104,1,Wind Direction - Resultant,1 HOUR,NA,2022-01-01,2022,1,Degrees Compass,NA,24,100,140.791667,195.00,15,.,Y
84313,1,Black carbon PM2.5 STP,1 HOUR,NA,2022-01-01,2022,1,Micrograms/cubic meter (25 C),NA,24,100,0.458333,1.25,1,.,Y
88101,3,PM2.5 - Local Conditions,24-HR BLK AVG,PM25 Annual 2012,2022-01-01,2022,1,Micrograms/cubic meter (LC),NA,1,100,7.100000,7.10,0,30,Y


In [222]:
# Convert string date to Date field type
df_2022_01_curated <- df_2022_01_curated %>%
    mutate(date_local = as.Date(date_local, format = "%Y-%m-%d"))

# See the updated data type
head(df_2022_01_curated)

parameter_code,poc,parameter_name,duration_description,pollutant_standard,date_local,year,day_in_year_local,units_of_measure,exceptional_data_type,observation_count,observation_percent,arithmetic_mean,first_maximum_value,first_maximum_hour,aqi,daily_criteria_indicator
<dbl>,<dbl>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
87101,1,"Particle Number, Total Count",1 HOUR,NA,2022-01-01,2022,1,Count per cm^3,NA,24,100,7062.208333,14300.00,17,.,Y
61107,1,Std Dev Vt Wind Direction,1 HOUR,NA,2022-01-01,2022,1,Degrees Compass,NA,24,100,17.166667,25.00,7,.,Y
62101,1,Outdoor Temperature,1 HOUR,NA,2022-01-01,2022,1,Degrees Fahrenheit,NA,24,100,48.958333,54.00,15,.,Y
61104,1,Wind Direction - Resultant,1 HOUR,NA,2022-01-01,2022,1,Degrees Compass,NA,24,100,140.791667,195.00,15,.,Y
84313,1,Black carbon PM2.5 STP,1 HOUR,NA,2022-01-01,2022,1,Micrograms/cubic meter (25 C),NA,24,100,0.458333,1.25,1,.,Y
88101,3,PM2.5 - Local Conditions,24-HR BLK AVG,PM25 Annual 2012,2022-01-01,2022,1,Micrograms/cubic meter (LC),NA,1,100,7.100000,7.10,0,30,Y


In [223]:
str(df_2022_01_curated)

tibble [743 × 17] (S3: tbl_df/tbl/data.frame)
 $ parameter_code          : num [1:743] 87101 61107 62101 61104 84313 ...
 $ poc                     : num [1:743] 1 1 1 1 1 3 3 3 1 1 ...
 $ parameter_name          : chr [1:743] "Particle Number, Total Count" "Std Dev Vt Wind Direction" "Outdoor Temperature" "Wind Direction - Resultant" ...
 $ duration_description    : chr [1:743] "1 HOUR" "1 HOUR" "1 HOUR" "1 HOUR" ...
 $ pollutant_standard      : chr [1:743] NA NA NA NA ...
 $ date_local              : Date[1:743], format: "2022-01-01" "2022-01-01" ...
 $ year                    : num [1:743] 2022 2022 2022 2022 2022 ...
 $ day_in_year_local       : num [1:743] 1 1 1 1 1 1 1 1 1 2 ...
 $ units_of_measure        : chr [1:743] "Count per cm^3" "Degrees Compass" "Degrees Fahrenheit" "Degrees Compass" ...
 $ exceptional_data_type   : logi [1:743] NA NA NA NA NA NA ...
 $ observation_count       : num [1:743] 24 24 24 24 24 1 1 24 24 24 ...
 $ observation_percent     : num [1:743] 100 100 1

Great! We've cut down the number of columns and converted the date field to a datetime format!
Next lets see how we can add more data from other files.

## 4. Concatenation
So far we've worked with one month's worth of data. Let's see how we can combine DataFrames together.

We will be using the [`bind_rows`](https://dplyr.tidyverse.org/reference/bind.html) function to combine two or more DataFrames.


In [224]:
# Read in Feburary data
df_2022_02 <- read_csv(file.path(path_to_monthly_data, 'daily_44_007_0022_2022_02.csv'))

df_2022_02_curated <- df_2022_02 %>%
    rename_with(to_snake_case) %>%
    select(all_of(keep_cols)) %>%
    mutate(date_local = as.Date(date_local, format = "%Y-%m-%d"))
    
head(df_2022_02_curated)

Rows: 699 Columns: 34
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (14): Datum, Parameter Name, Duration Description, Pollutant Standard, ...
dbl  (14): State Code, County Code, Site Number, Parameter Code, POC, Latitu...
lgl   (5): Exceptional Data Type, Nonreg Observation Count, Nonreg Arithmeti...
date  (1): Date (Local)

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


parameter_code,poc,parameter_name,duration_description,pollutant_standard,date_local,year,day_in_year_local,units_of_measure,exceptional_data_type,observation_count,observation_percent,arithmetic_mean,first_maximum_value,first_maximum_hour,aqi,daily_criteria_indicator
<dbl>,<dbl>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
61107,1,Std Dev Vt Wind Direction,1 HOUR,NA,2022-02-01,2022,32,Degrees Compass,NA,24,100,30.12500,44.0,17,.,Y
88101,3,PM2.5 - Local Conditions,1 HOUR,NA,2022-02-01,2022,32,Micrograms/cubic meter (LC),NA,12,50,18.33333,29.0,0,.,N
61104,1,Wind Direction - Resultant,1 HOUR,NA,2022-02-01,2022,32,Degrees Compass,NA,24,100,241.58333,357.0,12,.,Y
87101,1,"Particle Number, Total Count",1 HOUR,NA,2022-02-01,2022,32,Count per cm^3,NA,24,100,25241.33333,39175.0,7,.,Y
62101,1,Outdoor Temperature,1 HOUR,NA,2022-02-01,2022,32,Degrees Fahrenheit,NA,24,100,25.20833,34.0,13,.,Y
61103,1,Wind Speed - Resultant,1 HOUR,NA,2022-02-01,2022,32,Knots,NA,24,100,3.11250,4.4,14,.,Y


<div class="alert alert-block alert-info">
<b>Bind multiple data frames by row</b>

Bind any number of data frames by row, making a longer result. This is similar to do.call(rbind, dfs), but the output will contain all columns that appear in any of the inputs.

`bind_rows(..., .id=NULL)`

[Documentation Link](https://dplyr.tidyverse.org/reference/bind_rows.html)
</div>


In [225]:
# Concatenate
df_combined <- bind_rows(df_2022_01_curated, df_2022_02_curated)

In [226]:
# Observe dimensions
dim(df_2022_01_curated)
dim(df_2022_02_curated)
dim(df_combined)

[1] 743  17

[1] 699  17

[1] 1442   17

In [227]:
head(df_combined)
tail(df_combined)

parameter_code,poc,parameter_name,duration_description,pollutant_standard,date_local,year,day_in_year_local,units_of_measure,exceptional_data_type,observation_count,observation_percent,arithmetic_mean,first_maximum_value,first_maximum_hour,aqi,daily_criteria_indicator
<dbl>,<dbl>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
87101,1,"Particle Number, Total Count",1 HOUR,NA,2022-01-01,2022,1,Count per cm^3,NA,24,100,7062.208333,14300.00,17,.,Y
61107,1,Std Dev Vt Wind Direction,1 HOUR,NA,2022-01-01,2022,1,Degrees Compass,NA,24,100,17.166667,25.00,7,.,Y
62101,1,Outdoor Temperature,1 HOUR,NA,2022-01-01,2022,1,Degrees Fahrenheit,NA,24,100,48.958333,54.00,15,.,Y
61104,1,Wind Direction - Resultant,1 HOUR,NA,2022-01-01,2022,1,Degrees Compass,NA,24,100,140.791667,195.00,15,.,Y
84313,1,Black carbon PM2.5 STP,1 HOUR,NA,2022-01-01,2022,1,Micrograms/cubic meter (25 C),NA,24,100,0.458333,1.25,1,.,Y
88101,3,PM2.5 - Local Conditions,24-HR BLK AVG,PM25 Annual 2012,2022-01-01,2022,1,Micrograms/cubic meter (LC),NA,1,100,7.100000,7.10,0,30,Y


parameter_code,poc,parameter_name,duration_description,pollutant_standard,date_local,year,day_in_year_local,units_of_measure,exceptional_data_type,observation_count,observation_percent,arithmetic_mean,first_maximum_value,first_maximum_hour,aqi,daily_criteria_indicator
<dbl>,<dbl>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
43815,2,Ethylene dichloride,24 HOUR,NA,2022-02-28,2022,59,Parts per billion Carbon,NA,1,100,0.0,0.0,0,.,Y
43802,2,Dichloromethane,24 HOUR,NA,2022-02-28,2022,59,Parts per billion Carbon,NA,1,100,0.1,0.1,0,.,Y
43817,2,Tetrachloroethylene,24 HOUR,NA,2022-02-28,2022,59,Parts per billion Carbon,NA,1,100,0.0,0.0,0,.,Y
43824,2,Trichloroethylene,24 HOUR,NA,2022-02-28,2022,59,Parts per billion Carbon,NA,1,100,0.0,0.0,0,.,Y
45807,2,"1,4-Dichlorobenzene",24 HOUR,NA,2022-02-28,2022,59,Parts per billion Carbon,NA,1,100,0.0,0.0,0,.,Y
43372,2,Methyl tert-butyl ether,24 HOUR,NA,2022-02-28,2022,59,Parts per billion Carbon,NA,1,100,0.0,0.0,0,.,Y


### Concat all the files!
Now that we've learned how to concatenate files. Let's combine all the monthly data.
Doing it manually for each file would be cumbersome. So lets use a function!

We won't have time to walk through this function in detail, but we encourage you to take a look on your own time. It covers concepts of:
1) Defining a function with: arguments, defaults, and variable keyword arguments
2) Listing files in a directory
3) for loops
4) if/else constructs
5) What we just learned about reading csv and concatenation


In [228]:
#' Combine CSV Files
#'
#' Searches a directory for text files, imports as a data frame and concatenates
#' to a single data frame
#' 
#' @param path path to directory
#' @param prefix string prefix to search for
#' @param suffix string suffix to search for, option, default ".csv"
#'
#' @return dataFrame
#'
#' @examples
combine_csv_files <- function(path, prefix, suffix=".csv") {
    # List files in the directory
    list_files_in_path <- sort(list.files(path_to_monthly_data))

    # Initialize an empty list to store data frames
    list_df <- list()

    # Loop through files
    for (file in list_files_in_path) {
        # Check if the files starts with the prefix and ends with the suffix
        if (startsWith(file, prefix) && endsWith(file, suffix)) {
            # Read the CSV file
            temp_df <- read_csv(file.path(path, file))

            # Curate the data frame
            temp_df_curated <- temp_df %>%
                rename_with(to_snake_case) %>%
                select(all_of(keep_cols)) %>%
                mutate(date_local = as.Date(date_local, format = "%Y-%m-%d"))
            
            # Add to the list of data frames
            # When appending a data frame to a list, it needs to be wraped in `list()` to ensure
            # that is it treated as a single element within the list. Otherwise, R may try to 
            # append each column of data as separate elements
            list_df <- append(list_df, list(temp_df_curated))
        }
    }

    # Concatenate all of the data frames
    return(bind_rows(list_df))
}

In [229]:
# Use the function defined above
df_2022 <- combine_csv_files(path_to_monthly_data, prefix='daily_44_007_0022_2022_')

Rows: 743 Columns: 34
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (14): Datum, Parameter Name, Duration Description, Pollutant Standard, ...
dbl  (14): State Code, County Code, Site Number, Parameter Code, POC, Latitu...
lgl   (5): Exceptional Data Type, Nonreg Observation Count, Nonreg Arithmeti...
date  (1): Date (Local)

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 699 Columns: 34
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (14): Datum, Parameter Name, Duration Description, Pollutant Standard, ...
dbl  (14): State Code, County Code, Site Number, Parameter Code, POC, Latitu...
lgl   (5): Exceptional Data Type, Nonreg Observation Count, Nonreg Arithmeti...
date  (1): Date (Local)

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specif

In [230]:
dim(df_2022)

[1] 8591   17

In [231]:
head(df_2022)

parameter_code,poc,parameter_name,duration_description,pollutant_standard,date_local,year,day_in_year_local,units_of_measure,exceptional_data_type,observation_count,observation_percent,arithmetic_mean,first_maximum_value,first_maximum_hour,aqi,daily_criteria_indicator
<dbl>,<dbl>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
87101,1,"Particle Number, Total Count",1 HOUR,NA,2022-01-01,2022,1,Count per cm^3,NA,24,100,7062.208333,14300.00,17,.,Y
61107,1,Std Dev Vt Wind Direction,1 HOUR,NA,2022-01-01,2022,1,Degrees Compass,NA,24,100,17.166667,25.00,7,.,Y
62101,1,Outdoor Temperature,1 HOUR,NA,2022-01-01,2022,1,Degrees Fahrenheit,NA,24,100,48.958333,54.00,15,.,Y
61104,1,Wind Direction - Resultant,1 HOUR,NA,2022-01-01,2022,1,Degrees Compass,NA,24,100,140.791667,195.00,15,.,Y
84313,1,Black carbon PM2.5 STP,1 HOUR,NA,2022-01-01,2022,1,Micrograms/cubic meter (25 C),NA,24,100,0.458333,1.25,1,.,Y
88101,3,PM2.5 - Local Conditions,24-HR BLK AVG,PM25 Annual 2012,2022-01-01,2022,1,Micrograms/cubic meter (LC),NA,1,100,7.100000,7.10,0,30,Y


In [232]:
tail(df_2022)

parameter_code,poc,parameter_name,duration_description,pollutant_standard,date_local,year,day_in_year_local,units_of_measure,exceptional_data_type,observation_count,observation_percent,arithmetic_mean,first_maximum_value,first_maximum_hour,aqi,daily_criteria_indicator
<dbl>,<dbl>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
43826,2,"1,1-Dichloroethylene",24 HOUR,NA,2022-12-31,2022,365,Parts per billion Carbon,NA,1,100,0.00000,0.0,0,.,Y
43860,2,Vinyl chloride,24 HOUR,NA,2022-12-31,2022,365,Parts per billion Carbon,NA,1,100,0.00000,0.0,0,.,Y
43815,2,Ethylene dichloride,24 HOUR,NA,2022-12-31,2022,365,Parts per billion Carbon,NA,1,100,0.00000,0.0,0,.,Y
61107,1,Std Dev Vt Wind Direction,1 HOUR,NA,2022-12-31,2022,365,Degrees Compass,NA,24,100,17.16667,24.0,10,.,Y
43552,2,Methyl ethyl ketone,24 HOUR,NA,2022-12-31,2022,365,Parts per billion Carbon,NA,1,100,1.20000,1.2,0,.,Y
43372,2,Methyl tert-butyl ether,24 HOUR,NA,2022-12-31,2022,365,Parts per billion Carbon,NA,1,100,0.00000,0.0,0,.,Y


## 5. Renaming Columns
Before we jump into more detailed EDA we'll want to update the DataFrame to make our lives a bit easier. We'll be typing column names often to query the data so let's start by renaming the columns to a standard format.

* standard naming convention: snake_case
  * During our import, we have already converted the column names to snake_case using `rename_with(to_snake_case)`. 
* simplify complex names
  * We can reword some of the column names to make them more concise

In [233]:
# Column Names
colnames(df_2022)

[1] "parameter_code"           "poc"                     
 [3] "parameter_name"           "duration_description"    
 [5] "pollutant_standard"       "date_local"              
 [7] "year"                     "day_in_year_local"       
 [9] "units_of_measure"         "exceptional_data_type"   
[11] "observation_count"        "observation_percent"     
[13] "arithmetic_mean"          "first_maximum_value"     
[15] "first_maximum_hour"       "aqi"                     
[17] "daily_criteria_indicator"

There are two columns `date_local` and `day_in_year_local` that we can more concisely name as `date` and `day_in_year`. Let's use 

In [234]:
# Rename date_local to date
colnames(df_2022)[colnames(df_2022) == 'date_local'] <- 'date'

In [235]:
# Rename date_in_year_local to date_in_year
colnames(df_2022)[colnames(df_2022) == 'day_in_year_local'] <- 'day_in_year'

In [236]:
# Let's check! 
colnames(df_2022)

[1] "parameter_code"           "poc"                     
 [3] "parameter_name"           "duration_description"    
 [5] "pollutant_standard"       "date"                    
 [7] "year"                     "day_in_year"             
 [9] "units_of_measure"         "exceptional_data_type"   
[11] "observation_count"        "observation_percent"     
[13] "arithmetic_mean"          "first_maximum_value"     
[15] "first_maximum_hour"       "aqi"                     
[17] "daily_criteria_indicator"

Great! Now are column names are much more manageable.

## 6. Exploring Values
Let's start exploring the dataset.

Questions:
1. How complete is the data?
2. Are there duplicates?
3. How many parameters are measured?
4. How often are the parameters measured?
5. What are the descriptive stats of the numeral data?

We will cover the following topics:
1. Indexing
2. Checking for Nulls and Duplicates
3. Counts and Uniques
4. Querying
5. Descriptive Stats and groupby

### 6.1 Indexing
We often will need to look different slices of the dataframe.
* Slices are different groups of rows and/or columns.
* We perform indexing using brackets `[]`, similar to indexing lists.

```python
# For a single column
df_2022['parameter_name']

# For multiple columns, put column names into a vector
df_2022[c('parameter_name', 'date')]

# For a slice of rows (same as list indexing)
df_2022[5:10, ] # Note the comma. It indicates [rows, columns].

# For a slice of rows and columns
df_2022[5:10, c('parameter_name', 'parameter_code')]
```


In [237]:
df_2022[5:10, c('parameter_name', 'parameter_code')]

parameter_name,parameter_code
<chr>,<dbl>
Black carbon PM2.5 STP,84313
PM2.5 - Local Conditions,88101
PM2.5 - Local Conditions,88101
PM2.5 - Local Conditions,88101
Relative Humidity,62201
Black carbon PM2.5 STP,84313


### 6.2.0 Checking for Nulls
Nulls or NAs are values that represent no data or missing data.
* They are often represented as **NA** for "Not Available"
* The tidyverse readr package `read_csv` function fill NAs for blank values upon import.
* Be aware that scientific datasets often use large negative numbers outside of the normal range (like -999) to represent null data.

We can check for the number of null values quickly using the method `is.na()` and summing the results
* `is.na()` will create a boolean matrix and `colSum()` will sum by columns.


In [238]:
df_example <- data.frame(
    numbers = c(1, 2, 3, NA), 
    strings = c(NA, 'hello', 'world', NA)
)

df_example

numbers,strings
<dbl>,<chr>
1,NA
2,hello
3,world
NA,NA


In [239]:
# Example usage of is.na
df_example_na <- is.na(df_example)

df_example_na

numbers,strings
FALSE,TRUE
FALSE,FALSE
FALSE,FALSE
TRUE,TRUE


In [240]:
colSums(df_example_na)

numbers strings 
      1       2

In [241]:
# Now let's try our data
df_2022_na <- is.na(df_2022)
head(df_2022_na)

parameter_code,poc,parameter_name,duration_description,pollutant_standard,date,year,day_in_year,units_of_measure,exceptional_data_type,observation_count,observation_percent,arithmetic_mean,first_maximum_value,first_maximum_hour,aqi,daily_criteria_indicator
FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [242]:
# Count
colSums(df_2022_na)

parameter_code                      poc           parameter_name 
                       0                        0                        0 
    duration_description       pollutant_standard                     date 
                       0                     7787                        0 
                    year              day_in_year         units_of_measure 
                       0                        0                        0 
   exceptional_data_type        observation_count      observation_percent 
                    8591                        0                        0 
         arithmetic_mean      first_maximum_value       first_maximum_hour 
                       0                        0                        0 
                     aqi daily_criteria_indicator 
                       0                        0

Luckily this dataset does not have missing data in important fields. The fields with null values make sense.
* Not all chemicals measured have a regulatory standard that would be in the **pollutant_standard** field, only the most toxic.
* **exceptional_data_type** is a flag field for anomalous conditions and events.

### 6.2.1 Handling missing data
Missing data is handled on a case-by-case basis. Some options are: 
* Drop missing data
    * If you have enough data otherwise   
* Impute data
    * Method will depend what data you have available
    * Some examples
        1. Extrapolate small gaps in a timeseries
        2. Use mean or median values from similar data points
        3. Sampling from a probability distribution



### 6.3 Checking for Duplicates
We use the [`duplicated()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method to check for duplicates.

The `duplicated()` method generates a boolean array of indicating the duplicates based on a subset of keys.

Our intuition is that this dataset should have one record per day for each parameter. Let's check if that assumption is correct.

In [ ]:
# Get a boolean array of duplicates
mask_duplicates = df_2022.duplicated(subset=['date', 'parameter_code'], keep=False)
mask_duplicates

* We supplied duplicated method with the `date` and `parameter_code` as the subset of columns to act as a unique key.
* We also supply a keyword `keep=False`.
    * This logic is very confusing.
        * The default behavior of this method is to mark only duplicates after the first occurrence as the duplicates.
        * This makes it easy to "keep" the first occurrence and remove all subsequent duplicates.
        * By supplying `False` we are turning off that behavior because we want to consider all duplicates.

In [ ]:
any(mask_duplicates)

In [ ]:
mask_duplicates.sum()

OK. There are clearly some duplicates in this dataset. Let's take a closer look at these in the next section.

### 6.4 Querying and Sorting
By querying the dataframe we can better examine the duplicates we identified in the previous section.

There are two main ways to query a DataFrame:
1. Masking and Indexing
2. [`query()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) method

The choice depends on preference and context.
* Masking and Indxing is clearer for complex queries with multiple conditions.
* The `query()` method good for short and quick queries.

Syntax example:
```Python
# Masking and Indexing
mask_param = df_2022['parameter_name'] == 'PM2.5 - Local Conditions'
df_2022[mask_param]

# Query method
df_2022.query('parameter_name == "PM2.5 - Local Conditions"')
```

Conveniently we just created a mask of duplicate values in the last section so let's examine that.
We will also use the [`sort_values()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) method to order the DataFrame.

In [ ]:
# Masking and Indexing
df_2022[mask_duplicates].sort_values(by=['date', 'parameter_name'])

The PM2.5 measurment seems to have something different going on than the other duplicates. Let's take a look at that parameter first.

In [ ]:
(df_2022.loc[mask_duplicates, ['date', 'parameter_name', 'duration_description', 'pollutant_standard', 'arithmetic_mean', 'first_maximum_value', 'first_maximum_hour','aqi']]
 .sort_values(by=['date', 'parameter_name', 'duration_description', 'pollutant_standard'])
 .head(12))

*What do we notice about the PM2.5 measurement?*
* There are 3 records per day. They differ in duration_description and pollutant_standard.
* The "1 HOUR" duration seems to have the most accurate information for our purposes because it has more "arithmetic_mean" precision, and better information about the first maximum value and hour over the day.
* The "1 Hour" duration records does not have Air Quality Index "aqi" information

**Conclusion**
* We should keep only the "1 HOUR" duration and remove the others.
* We'll save the other durations in a separate DataFrame in the case we want the daily AQI information.

We can do this with querying!

In [ ]:
# Create a mask for the values we want to remove
mask_remove = (df_2022['parameter_name'] == 'PM2.5 - Local Conditions') & (df_2022['duration_description'] != '1 HOUR')

# Create dataframe of PM2.5 records we remove
df_removed_records = df_2022[mask_remove].reset_index(drop=True)

# Create dataframe of everything we want to keep. The NOT (~) operator reverses the mask.
df_2022_cleaned_v1 = df_2022[~mask_remove].reset_index(drop=True)

In [ ]:
df_2022_cleaned_v1.query("parameter_name == 'PM2.5 - Local Conditions'").head(10)

Now let's return to look at those other duplicate parameters. Below we will combine two masks. Note that we are referencing the pre-cleaned dataframe.

In [ ]:
# This is an example of combining two masks. The duplicate mask and the not PM2.5 mask
mask_not_pm2p5 = df_2022['parameter_name'] != 'PM2.5 - Local Conditions'
df_2022[mask_duplicates & mask_not_pm2p5].sort_values(by=['date', 'parameter_name'])

*What do we notice here?*
* It appears these duplicates are pairs with a different 'poc' code.
    * The 'poc' code is an identifier for sampling instruments at the site. These parameters are sampled in replicates on separate instruments as a quality-control check. If replicate results differ too much, it may signal sampling error.
* At this point we have several options:
    1. Drop one of the duplicates
    2. Average the duplicates
    3. Leave the data alone

We'll choose option 3 for now because the duplicate treatment may depend on the type of analysis being performed. The decision can be made at that point.

### 6.5 Counts and Uniques
Counts and unique values are great for looking at categorical data.

In [ ]:
# Get the count of unique values in each column
df_2022_cleaned_v1.nunique()

In [ ]:
# Get a list of unique values in a column
df_2022_cleaned_v1['parameter_name'].unique()

In [ ]:
# Get the counts of each parameter
counts = df_2022_cleaned_v1['parameter_name'].value_counts()
counts

The output is abbreviated. To see specific analytes you need index with a list.

In [ ]:
counts[['PM2.5 - Local Conditions', 'Lead PM10 STP', 'Acetaldehyde', 'Cyclohexane']]

In [ ]:
counts.head(10)

### 6.5 Descriptive Stats and groupby
The method `.describe()` is a great way to get descriptive statistics on numerical columns. When called on the dataframe, it will automatically provide statistics on each numerical column.

In [ ]:
df_2022_cleaned_v1.describe()

Though this isn't very useful for our dataset at the moment because it's creating statistics across 90+ parameters. It would be better to chunk the DataFrame by parameter and run describe. This is where a powerful method called `groupby` is useful.

In [ ]:
# Create a Groupby object with parameter_name
# This object chunks the dataframe by parameter_name
gb_param = df_2022_cleaned_v1.groupby(by='parameter_name')
gb_param['arithmetic_mean'].describe()

In [ ]:
# To look at the PM2.5 stats
df_gb_stats = gb_param['arithmetic_mean'].describe()
df_gb_stats.loc['PM2.5 - Local Conditions']

You can groupby multiple columns. Here's an example of sample counts by parameter and month.

In [ ]:
# We group by parameter name and the data column converted to the month number
gb_param_month = df_2022_cleaned_v1.groupby(by=['parameter_name', df_2022_cleaned_v1['date'].dt.month])
df_param_monthly_counts = gb_param_month['arithmetic_mean'].count()  # Count the number of records
df_param_monthly_counts.loc[['PM2.5 - Local Conditions', '1,1,1,2-Tetrachloroethane']]  # View two specific parameters

We can see that the frequency of sampling between these parameters is different.

Previously we found that some parameters had duplicate values. We can use groupby to average and count the values.

In [ ]:
gb_param_date = df_2022_cleaned_v1.groupby(by=['parameter_name', 'date'])
(gb_param_date['arithmetic_mean']
 .agg(['mean', 'count'])  # We use the aggregate method to apply both 'mean' and 'count'
 .query("count > 1")  # Query for only those with more than one record
 )

## 7. Exporting DataFrames
We made some interesting groupby tables with summary stats in the last section. Let's try exporting them so we can view in another application or import them in a future analysis.

We use the `to_csv()` method to export to csv.
See the [Pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) to see all the other formats you can export to.

In [ ]:
# Create a Path object to the reports folder
outputs_path = Path('..', 'reports')

# Save the groupby parameters and descriptive stats of the arithmetic mean to a DataFrame
df_gb_param_mean_describe = gb_param['arithmetic_mean'].describe()

# Call the to_csv method on the dataframe specifying path and filename
df_gb_param_mean_describe.to_csv(outputs_path / 'stats-arithmetic_mean-by-param.csv')

Navigate to the reports directory and confirm that a file was written.

## 8. Merging
If we wanted to join two tables based on common key we would use the [`merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) method.

There is a file with parameter classes that categorizes the parameters. This might help us make sense of which parameters are related to each other. Let's join it to our dataframe!
```
project
├── data
│   └── raw
│       └── params_class.csv
```

In [ ]:
df_param_classes = pd.read_csv(data_path / 'raw' / 'params_class.csv')
df_param_classes.head()

This file has two new columns:
1. **param_class** - The parameter classification
2. **tier1** - A boolean field for EPA designated Tier 1 major risk toxins.
    * Note that this does not apply to particulate matter which are criteria pollutants but are not designated as toxins.

In [ ]:
# Count of the number of parameters in each class
df_param_classes['param_class'].value_counts()

In [ ]:
# Query the 18 Tier 1 toxins
df_param_classes.query("tier1 == True")

In [ ]:
# Merge the dataframes using parameter_code as the key with a left-join
df_2022_cleaned_v2 = df_2022_cleaned_v1.merge(df_param_classes, on='parameter_code', how='left')
df_2022_cleaned_v2.head()

We used a left-join here. This means that only keys from the left table matter in the join. Keys that are in the right table but not the left will not be joined.

Notice that parameter_name is duplicated with an appended "_x" and "_y" because the column existed in both tables. You can prevent this by indexing only the columns you want to join. For now we are just going to drop and rename the columns for an example of the `drop()` method!

In [ ]:
df_2022_cleaned_v3 = (df_2022_cleaned_v2
                      .drop(columns='parameter_name_y')  # Remove the column
                      .rename(columns={'parameter_name_x': 'parameter_name'})  # We've seen this method previously!
                      )
df_2022_cleaned_v3.columns

Great! Lets save a copy of our processed dataset to the processed directory of the data folder.
This is good practice so that future analysis can start from a cleaner version of the data.

In [ ]:
df_2022_cleaned_v3.to_csv(data_path / 'processed' / 'df_2022_processed.csv', index=False)

## 9. Reshaping and Plotting

We will be plotting with Pandas' built-in plotting methods and functions.
* Pandas plotting is built as a front-end to the Matplotlib package.
* The plotting functionalities are bare-bones but are good for quick EDA.
* There are many other plotting packages out there here are a few:
    * [matplotlib](https://matplotlib.org) - The cornerstone of several plotting libraries, including pandas. It's hard to learn but necessary for fine-tuning figures.
    * [seaborn](https://seaborn.pydata.org) - Based on matplotlib but with a higher-level API easier to start with.
    * [plotnine](https://plotnine.readthedocs.io/en/stable/) - Implementation of R's popular ggplot2 in python. Also based matplotlib.
    * [vega-altair](https://altair-viz.github.io) - Declarative graphing library built upon vega-lite grammar. Great for those with JavaScript background.
    * [plotly](https://plotly.com/python/) - For interactive graphics
    * [bokeh](https://bokeh.org) -Also for interactive graphics


### Reshaping
Plotting will require many dataframe transformations to shape the dataframe into a plot-able structure.
* With timeseries data we typically want a structure where rows are a single date/time and plot values are along columns.

| Date | param1 | param_2 | ...   | param_n |
|-----|-------|-------|-------|-------|
| Date1 | value | value | value | value |
| Date2 | value | value | value | value |
| ... | value | value | value | value |

The [`pd.pivot_table()`](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) function will help us reshape the table to this format.

We will only look at particulate matter for plotting.

In [ ]:
df_2022_c = df_2022_cleaned_v3 # saving as a shorter name for convenience

mask_pm = (df_2022_c['param_class'] == 'particulate')  # mask for just the particulate parameters
df_pm = pd.pivot_table(df_2022_c[mask_pm],
                       values='arithmetic_mean',  # The column we want as values
                       index='date',  # Set the date as the index of the new dataframe
                       columns='parameter_name'  # The column with labels we wish to appear as columns
                       )
df_pm.columns = ['Black Carbon', 'PM10', 'PM2.5', 'Particle Number']  # Rename columns for convenience
df_pm.head()

Now we have a wide table with the index as dates and parameters as columns! Let's begin plotting!

### Plotting
1. Boxplots - Distribution and outliers
2. Time Series - Temporal trends
3. Scatter - Relationships between variables
4. Autocorrelation & Lag Plots - Relationships with previous measurements

Please check out panda's [Chart visualization](https://pandas.pydata.org/docs/user_guide/visualization.html) documentation for many more plot types.

### 9.1 Boxplots
Boxplots can be generated by either then [`boxplot()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.boxplot.html#matplotlib.axes.Axes.boxplot) or [`plot.box()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.box.html) methods. They are aliases.


In [ ]:
# We'll first drop the "Particle Number" parameter because it's a completely different measure and magnitude.
df_pm_nopn = df_pm.drop(columns=['Particle Number'])
df_pm_nopn.plot.box(ylabel='ug/m3');

We should probably plot the parameters on their own panel due the differences in magnitude.

In [ ]:
df_pm_nopn.plot.box(subplots=True,  # Plot each column in on its own panel
                    ylabel='ug/m3');
plt.subplots_adjust(wspace=.5)  # Adjust space between subplots so they don't overlap

Seems like there are outliers that may be worth removing in future analysis.
Collecting more years of data could give us a better idea of these are true outliers.

### 9.2 Time Series
Time series are useful to look at temporal trends.
To plot a time series we simply call the method [`plot()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html).

Our dataframe's index is set to the date so the plot function will assume that as the x-axis.

In [ ]:
# Plot a time series
df_pm_nopn.plot();

PM10 did not show up because it has a lot of NaNs. We can plot the data points to see this by providing keyword arguments to the plot command. Here we add many other keyword arguments.

See the [`plot()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html) documentation for other options.

In [ ]:
ax = df_pm_nopn.plot(marker='o',  # Add circle markers
                     ylabel='ug/m3',  # Adds y-axis label
                     logy=True,  # Converts y-axis to a log-scal
                     alpha=.6,   # Transparency scale from 0-1, 0 being invisible
                     figsize=(12, 5));  # width and height of the figure

# Add a dashed line for PM2.5 criteria value at 9 ug/m3
ax.axhline(9, linestyle='--', color='gray');

Environmental data can be noisy. Applying a rolling mean is a technique to smooth-out noise and look at underlying trends.
We use the [`rolling()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html) method to the DataFrame to calculate this.

In [ ]:
df_pm_nopn_rm5 = df_pm_nopn.drop(columns=['PM10']).rolling(5).mean()  # Compute a 5-day rolling average
df_pm_nopn_rm5.plot(marker='o',  # Add circle markers
                    ylabel='ug/m3',  # Adds y-axis label
                    alpha=.6,   # Transparency scale from 0-1, 0 being invisible
                    subplots=True,
                    figsize=(12, 5));

* There's not enough data to compute a rolling average for PM10.
* Black carbon and PM2.5 seem paired. This makes sense because Black carbon PM2.5 is a subset of PM2.5.
* There seems to be some temporal trends.

### 9.3 Scatter
Scatter plots help determine relationship between two variables.
Let's plot a scatter plot between PM2.5 and Black Carbon. We can specify this with the [`plot.scatter()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.scatter.html#pandas.DataFrame.plot.scatter) method.

In [ ]:
df_pm.plot.scatter(x='PM2.5', y='Black Carbon');

A [`scatter_matrix`](https://pandas.pydata.org/docs/reference/api/pandas.plotting.scatter_matrix.html) function is a quick method to look at relationships between all parameters simultaneously.

Note that this a function and not a method called on the DataFrame. We pass the DataFrame as an argument into the function.

In [ ]:
from pandas.plotting import scatter_matrix
df_pm_renamed = df_pm.copy()
scatter_matrix(df_pm, figsize=(6,6));

We can see that the parameters are generally positively correlated.

### 9.4 Autocorrelation & Lag Plots
Autocorrelation and lag plots help us determine if there are relationship with a measurement and previous measurements (lags).
Autocorrelation can find periodic trends in the data like seasonality.

pandas [`autocorrelation_plot`](https://pandas.pydata.org/docs/reference/api/pandas.plotting.autocorrelation_plot.html) function computes the correlation across all the lags for a set of measurements.

In [ ]:
from pandas.plotting import autocorrelation_plot, lag_plot

# We pass a single column into an autocorrelation plot function.
# The column cannot have null values. We call the method `fillna` to backward-fill null values with the subsequent values. This isn't an issue because there are few nulls.
autocorrelation_plot(df_pm_renamed['PM2.5'].fillna(method="bfill"))

In [ ]:
autocorrelation_plot(df_pm_renamed['PM2.5'].fillna(method="ffill")).set_xlim(1,20)

The horizontal lines correspond to the 99 (dashed) and 95 (solid) confidence bands. When the line is outside the bands the correlation is more confidently non-zero. The lags represent days. There is a steep drop-off in correlation after the first day.

We can look at the scatter plots with specific lags with the [`lag_plot`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.plotting.lag_plot.html) function.

Below we plot progressively larger lags. The plotting method below is an example of matplotlib syntax.

In [ ]:
# Create figure with 4 subplots. The fig variable is the object for configuring the entire figure (all subplots) the axs variable is an array with 4 axis objects corresponding to each subplot.
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(6,6))
axs = axs.ravel()  # Flatten the array
# Plot each lag and specify which subpanel it should go to with the `ax` keyword
lag_plot(df_pm_renamed['PM2.5'], lag=1, ax=axs[0]);
lag_plot(df_pm_renamed['PM2.5'], lag=2, ax=axs[1]);
lag_plot(df_pm_renamed['PM2.5'], lag=4, ax=axs[2]);
lag_plot(df_pm_renamed['PM2.5'], lag=12, ax=axs[3]);
plt.tight_layout()  # Adjust the subplot layout so they don't overlap


### 9.5 Resampling a timeseries 
In time series analysis you'll often be faced with data sampled at different scales (hourly, daily, weekly, etc.). Resampling is a method of __upscaling__ or __downscaling__ timeseries data.
* __upscaling__ - Make more datapoints by moving to a finer-scale sample (days -> hours). New data can be imputed or left as NaNs.
* __downscaling__ - Reduce datapoints by moving to a coaser-scale sample (hours -> days). Requires an aggregation metric like mean or sum.
  
Methods we'll use:
* `resample()` - [Documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html#pandas.DataFrame.resample)
    * Must supply a valid [*Date Offset*](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) as an argument

In [ ]:
df_pm.head()

In [ ]:
# Downscale from daily to monthly means using Month Start (MS) 
df_pm.resample('MS').mean()

In [ ]:
# Downscale from daily to 7-day 
df_pm.resample('7D').max().head()

In [ ]:
# Upscale from daily to 12-hour, no fill 
df_pm.resample('12h').asfreq().head(6)

In [ ]:
# Upscale from daily to 12-hour, with forward-fill
df_pm.resample('12h').asfreq().ffill().head()